In [28]:
headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "sec-ch-ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
}

### 리스트 추출 - 총 2705개여서 1000개씩 3번 수동으로 호출해서 파일 저장함

In [107]:
# 3개 리스트 추출
import requests
body = "{\"eventName\":\"\",\"sapCode\":\"J037024\",\"eventCode\":\"\",\"searchText\":\"\",\"brand\":\"\",\"selectedHierarchicalProperties\":[],\"selectedProperties\":[],\"selectedSectors\":[],\"selectedHomeOds\":[],\"selectedCountries\":[],\"filter\":\"ONLY_EXHIBITORS\",\"searchOrder\":\"BY_RELEVANCE\",\"language\":\"en_GB\",\"maxTextLength\":500,\"selectedMultiEvents\":[],\"createCache\":0}"
response = requests.post("https://ecatalogueusearch-api.firabarcelona.com/v1/us/unifiedSearch?page=0&size=1000&language=en_GB", headers=headers, data=body)
response


<Response [200]>

In [108]:
j_result = response.json()
print(len(j_result["list"]))

import json
with open('./output/ecatalogue_list_json.json', '+wt') as f:
    json.dump(j_result, f)

1000


In [ ]:
fetch("https://ecatalogueusearch-api.firabarcelona.com/v1/us/unifiedSearch?page=0&size=10&language=en_GB", {
  "headers": {
    "accept": "application/json, text/plain, */*",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "sec-ch-ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
  },
  "referrer": "https://ecatalogue.firabarcelona.com/",
  "referrerPolicy": "strict-origin-when-cross-origin",
  "body": "{\"eventName\":\"\",\"sapCode\":\"J037024\",\"eventCode\":\"\",\"searchText\":\"\",\"brand\":\"\",\"selectedHierarchicalProperties\":[],\"selectedProperties\":[],\"selectedSectors\":[],\"selectedHomeOds\":[],\"selectedCountries\":[],\"filter\":\"ONLY_EXHIBITORS\",\"searchOrder\":\"BY_RELEVANCE\",\"language\":\"en_GB\",\"maxTextLength\":500,\"selectedMultiEvents\":[],\"createCache\":0}",
  "method": "POST",
  "mode": "cors",
  "credentials": "omit"
});

In [ ]:
response = requests.get("https://ecatalogue-api.firabarcelona.com/v1/catalogues/133/countriesInUse?language=en_GB", headers=headers)
print(response.status_code)
country_json = response.json()
country_json

country_dict = {}
for item in country_json["_embedded"]["countries"]:
    country_dict[item["id"]] = item["name"]

country_dict

### 상세 내역 호출 및 적재

In [19]:
with open('./output/ecatalogue_list_json_001.json', 'rt') as f:
    j_result = json.load(f)

In [109]:
import pytz
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry

requests.adapters.DEFAULT_POOLSIZE = 100
retries = Retry(total=20)
# 세션을 생성하고 ConnectionPool 크기를 설정
session = requests.Session()
session.mount('https://', requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=retries))

# soup = BeautifulSoup(html, 'html.parser')
# items = soup.find_all('option')

def new_session():
    global session
    session = requests.Session()
    session.mount('https://', requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=retries))


def get_base_info(entity_id):
    response = session.get(f"https://ecatalogue-api.firabarcelona.com/v1/exhibitors/{entity_id}?projection=detail&language=en_GB")
    response.raise_for_status()
    if response.status_code != 200:
        raise Exception("** not 200")
    j_sub = response.json()
    return j_sub

def get_category_info(entity_id):
    response = session.get(f"https://ecatalogue-api.firabarcelona.com/v1/catalogues/133/exhibitors/{entity_id}/hierarchicalProperties?language=en_GB")
    response.raise_for_status()
    if response.status_code != 200:
        raise Exception("** not 200")
    j_sub = response.json()
    return j_sub

def add_category(item, item_list, str_list):
    for trans in item.get("translations", []):
        if trans["language"] == "en_GB":
            key = trans["name"]

    str_list.append(key)  
    if item.get("childHierarchicalProperties", []):
        for sub_item in item.get("childHierarchicalProperties", []):
            add_category(sub_item, item_list, str_list.copy())
    else:
        item_list.append(str_list)

In [114]:
import pandas as pd
import time
fail_dict = {}
result = []
def run(order_id):
    with open(f'./output/ecatalogue_list_json_00{order_id}.json', 'rt') as f:
        j_result = json.load(f)

    for index, row in enumerate(j_result['list']):
        try:
            id = row["entityId"]
            time.sleep(0.3)
            try:
                j_sub = get_base_info(id)
            except Exception as e:
                new_session()
                time.sleep(5)
                j_sub = get_base_info(id)
            result_item = {
                "name": j_sub["name"],
                "address": f'{j_sub["addressLine1"] or ""} {j_sub["addressLine2"] or ""}, {j_sub["zipCode"] or ""}, {j_sub["city"] or ""}, {country_dict.get(j_sub["countryId"], "")}',
                "website": j_sub["webSite"],
                "email": j_sub["email"],
                "contactEmail": j_sub["contactEmail"],
                "contactName": j_sub["contactName"],
                "contactTelephone": j_sub["contactTelephone"]
            }
            
            time.sleep(0.3)
            try :
                j_sub = get_category_info(id)
                if isinstance(j_sub, list) and j_sub and j_sub[0].get('childHierarchicalProperties'):
                    category_dict = {}

                    for item in j_sub:
                        item_list = []
                        add_category(item, item_list, [])

                        for tmp in item_list:
                            key = tmp.pop(0)
                            if "Product category" == key:
                                key = f"category - {key} - {tmp.pop(0)}"
                            else:
                                key = f"category - {key}"
                            category_dict.setdefault(key, [])
                            category_dict[key].append(" - ".join(tmp))
                    result_item.update(category_dict)
            except :
                pass
        except Exception as e:
            print(e)
            fail_dict[f"{order_id}-{index}"] = row

        result.append(result_item)
        if (index % 100) == (100 - 1):
            with open(f'./output/ecatalogue_result_00{order_id}.json', '+wt') as fp:
                json.dump(result, fp)
        
    with open(f'./output/ecatalogue_result_00{order_id}.json', '+wt') as fp:
        json.dump(result, fp)

run(1)
run(2)
run(3)

name 'new_session' is not defined


In [134]:
fail_dict

{'2-947': {'exhibitorId': None,
  'entityId': 219444,
  'type': 'EXHIBITOR',
  'weight': 0,
  'sponsoredByCategories': None,
  'sponsoredBySector': False,
  'sponsoredInHomePage': False,
  'featured': False,
  'brands': None,
  'exhibitorName': 'OXFAM INTERMON',
  'productTranslation': None,
  'exhibitorDescription': [],
  'exhibitorHierarchicalProperties': [],
  'productHierarchicalProperties': None,
  'properties': [],
  'odsProperties': None,
  'mainImage': None,
  'novelty': None,
  'isFavorite': None,
  'favoriteId': None,
  'note': None,
  'sectorId': 1394,
  'countryId': 209,
  'standLocation': 'GV/P7/0/F/315',
  'haveEmail': True,
  'eventSapCode': 'J037024',
  'orderId': '0001085524'}}

In [139]:
j_result["list"][947]
get_base_info(219444)

HTTPError: 403 Client Error:  for url: https://ecatalogue-api.firabarcelona.com/v1/exhibitors/219444?projection=detail&language=en_GB

In [135]:
#result_list = []
with open(f'./output/ecatalogue_list_json_002.json', 'rt') as f:
    j_result = json.load(f)
    print(len(j_result))
#result_list.extend(j_result)

10


In [133]:
df = pd.DataFrame(j_result)
df.to_csv('./output/ecatalogue_20240227_001.csv', index=False, quoting=1)

In [ ]:
fetch("https://ecatalogue-api.firabarcelona.com/v1/exhibitors/214747?projection=detail&language=en_GB", {
  "headers": {
    "accept": "application/json, text/plain, */*",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "sec-ch-ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site"
  },
  "referrer": "https://ecatalogue.firabarcelona.com/",
  "referrerPolicy": "strict-origin-when-cross-origin",
  "body": null,
  "method": "GET",
  "mode": "cors",
  "credentials": "omit"
});

In [ ]:
fetch("https://ecatalogue-api.firabarcelona.com/v1/catalogues/133/exhibitors/214747/hierarchicalProperties?language=en_GB", {
  "headers": {
    "accept": "application/json, text/plain, */*",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "sec-ch-ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site"
  },
  "referrer": "https://ecatalogue.firabarcelona.com/",
  "referrerPolicy": "strict-origin-when-cross-origin",
  "body": null,
  "method": "GET",
  "mode": "cors",
  "credentials": "omit"
});